In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
df1=pd.read_csv('../../data/dataset.csv')
mask=df1['F']==3
df=df1[~mask].copy()
halide_groups = df['Label'].apply(lambda x: 'Chloride' if 'Chloride' in x else 'Bromide' if 'Bromide' in x else 'Iodide')
df['X_group'] = halide_groups

# Example: Create box plots for defect energies grouped by cation/anion families
cation_families = df['Label'].apply(lambda x: 'Germanium' if 'Germanium' in x else 'Tin' if 'Tin' in x else 'Lead')
df['B_group'] = cation_families

In [ ]:
# Define the regular expression patterns for each ammonium family
patterns = {
    'Ammonium': r'Ammonium',
    'Methylammonium': r'Methylammonium',
    'Dimethylammonium': r'Dimethylammonium',
    'Trimethylammonium': r'Trimethylammonium',
    'Tetramethylammonium': r'Tetramethylammonium',
    'Ethylammonium': r'Ethylammonium',
    'Propylammonium': r'Propylammonium',
    'Isopropylammonium': r'Isopropylammonium',
    'Butylammonium': r'Butylammonium',
    'Hydroxylammonium': r'Hydroxylammonium',
    'Formamidinium': r'Formamidinium',
    'Acetamidinium': r'Acetamidinium',
    'Hydrazinium': r'Hydrazinium',
    'Guanidinium': r'Guanidinium',
    'Azetidinium': r'Azetidinium',
    'Imidazolium': r'Imidazolium'
}

# Function to map the Label to the ammonium family using regex
def map_ammonium_family(label):
    for family, pattern in patterns.items():
        if re.search(pattern, label):
            return family
    return 'Unknown'  # Default if no match is found

# Apply the mapping function to the 'Label' column
df['A_group'] = df['Label'].apply(map_ammonium_family)

In [ ]:
# Creating a new column to represent combinations of A-site and X-site families
df['A_X_combination'] = df['A_group'] + '_' + df['X_group']
df['A_B_combination'] = df['A_group'] + '_' + df['B_group']
df['B_X_combination'] = df['B_group'] + '_' + df['X_group']

In [ ]:


# Define the columns to analyze for ANOVA
columns_to_analyze = [
    'A SITE DFE', 
    'B SITE DFE', 
    'X SITE DFE', 
    'Bandgap, GGA (eV)'
]

# Define the groupings for the ANOVA
group_columns = ['A_group', 'A_X_combination', 'B_X_combination', 'A_B_combination']

# Loop through the group columns and the columns to analyze
for group_col in group_columns:
    for col in columns_to_analyze:
        # Perform One-Way ANOVA
        anova_result = stats.f_oneway(
            *[df[df[group_col] == group][col].dropna() 
              for group in df[group_col].unique()]
        )
        
        # Print the results
        print(f"One-Way ANOVA for {group_col} and {col}: F={anova_result.statistic}")

In [ ]:
# Loop through the group columns and the columns to analyze
for group_col in group_columns:
    for col in columns_to_analyze:
        try:
            tukey = pairwise_tukeyhsd(
                endog=df[col].dropna(),
                groups=df[group_col][df[col].notna()],
                alpha=0.05
            )
            print(f"Tukey HSD for {group_col} and {col}:")
            print(tukey.summary())
            print("\n" + "-"*80 + "\n")
        except Exception as e:
            print(f"Could not perform Tukey HSD for {group_col} and {col}: {e}")